In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb
#import xgboost as xgb
from scipy.sparse import vstack, csr_matrix, save_npz, load_npz
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import roc_auc_score
import gc
gc.enable()

In [0]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'category',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [0]:
%%time
train=pd.read_csv('/content/drive/My Drive/Microsoft Malware/train.csv',dtype=dtypes,low_memory=True)
train.shape
gc.collect()

CPU times: user 2min 23s, sys: 5.63 s, total: 2min 28s
Wall time: 2min 35s


In [0]:
Ind1=['IsProtected', 'Census_IsAlwaysOnAlwaysConnectedCapable',
       'Census_FlightRing', 'Census_HasOpticalDiskDrive',
       'Census_OSArchitecture', 'Census_GenuineStateName',
       'Census_IsTouchEnabled', 'Census_InternalBatteryType', 'SmartScreen',
       'Wdft_IsGamer', 'AVProductsInstalled', 'Census_PowerPlatformRoleName',
       'OrganizationIdentifier', 'AVProductStatesIdentifier',
       'Census_PrimaryDiskTypeName', 'Census_MDC2FormFactor',
       'Census_ProcessorCoreCount', 'Census_ChassisTypeName',
       'Census_InternalBatteryNumberOfCharges', 'AppVersion',
       'Census_ActivationChannel', 'Census_IsSecureBootEnabled',
       'Census_InternalPrimaryDisplayResolutionHorizontal',
       'Census_TotalPhysicalRAM', 'Census_OSBranch',
       'Census_OSWUAutoUpdateOptionsName', 'OsPlatformSubRelease',
       'OsBuildLab', 'Census_OSEdition', 'Census_OSUILocaleIdentifier',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches',
       'Census_PrimaryDiskTotalCapacity',
       'Census_FirmwareManufacturerIdentifier', 'Census_OSInstallTypeName',
       'LocaleEnglishNameIdentifier', 'Wdft_RegionIdentifier',
       'GeoNameIdentifier', 'Census_OSBuildRevision', 'Census_OSVersion',
       'Census_OEMNameIdentifier', 'CountryIdentifier',
       'Census_OEMModelIdentifier', 'Census_ProcessorModelIdentifier',
       'AvSigVersion', 'CityIdentifier', 'Census_FirmwareVersionIdentifier',
       'Census_SystemVolumeTotalCapacity']

In [0]:
trainy=train.loc[:,'HasDetections']

In [0]:
train =train.loc[:,Ind1]

In [0]:
train['MachineIdentifier'] = train.index.astype('uint32')
gc.collect()

334

In [0]:
%%time
test=pd.read_csv('/content/drive/My Drive/Microsoft Malware/test.csv',dtype=dtypes,low_memory=True)
test.shape
gc.collect()

CPU times: user 2min 9s, sys: 4.45 s, total: 2min 14s
Wall time: 2min 20s


In [0]:
Ind2=['IsProtected', 'Census_IsAlwaysOnAlwaysConnectedCapable',
       'Census_FlightRing', 'Census_HasOpticalDiskDrive',
       'Census_OSArchitecture', 'Census_GenuineStateName',
       'Census_IsTouchEnabled', 'Census_InternalBatteryType', 'SmartScreen',
       'Wdft_IsGamer', 'AVProductsInstalled', 'Census_PowerPlatformRoleName',
       'OrganizationIdentifier', 'AVProductStatesIdentifier',
       'Census_PrimaryDiskTypeName', 'Census_MDC2FormFactor',
       'Census_ProcessorCoreCount', 'Census_ChassisTypeName',
       'Census_InternalBatteryNumberOfCharges', 'AppVersion',
       'Census_ActivationChannel', 'Census_IsSecureBootEnabled',
       'Census_InternalPrimaryDisplayResolutionHorizontal',
       'Census_TotalPhysicalRAM', 'Census_OSBranch',
       'Census_OSWUAutoUpdateOptionsName', 'OsPlatformSubRelease',
       'OsBuildLab', 'Census_OSEdition', 'Census_OSUILocaleIdentifier',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches',
       'Census_PrimaryDiskTotalCapacity',
       'Census_FirmwareManufacturerIdentifier', 'Census_OSInstallTypeName',
       'LocaleEnglishNameIdentifier', 'Wdft_RegionIdentifier',
       'GeoNameIdentifier', 'Census_OSBuildRevision', 'Census_OSVersion',
       'Census_OEMNameIdentifier', 'CountryIdentifier',
       'Census_OEMModelIdentifier', 'Census_ProcessorModelIdentifier',
       'AvSigVersion', 'CityIdentifier', 'Census_FirmwareVersionIdentifier',
       'Census_SystemVolumeTotalCapacity']

In [0]:
test=test.loc[:,Ind2]
test['MachineIdentifier']  = test.index.astype('uint32')

In [0]:
gc.collect()

340

In [0]:
for usecol in train.columns.tolist()[1:-1]:

    train[usecol] = train[usecol].astype('str')
    test[usecol] = test[usecol].astype('str')
    
    #Fit LabelEncoder
    le = LabelEncoder().fit(
            np.unique(train[usecol].unique().tolist()+
                      test[usecol].unique().tolist()))

    #At the end 0 will be used for dropped values
    train[usecol] = le.transform(train[usecol])+1
    test[usecol]  = le.transform(test[usecol])+1

    agg_tr = (train
              .groupby([usecol])
              .aggregate({'MachineIdentifier':'count'})
              .reset_index()
              .rename({'MachineIdentifier':'Train'}, axis=1))
    agg_te = (test
              .groupby([usecol])
              .aggregate({'MachineIdentifier':'count'})
              .reset_index()
              .rename({'MachineIdentifier':'Test'}, axis=1))

    agg = pd.merge(agg_tr, agg_te, on=usecol, how='outer').replace(np.nan, 0)
    #Select values with more than 1000 observations
    agg = agg[(agg['Train'] > 1000)].reset_index(drop=True)
    agg['Total'] = agg['Train'] + agg['Test']
    #Drop unbalanced values
    agg = agg[(agg['Train'] / agg['Total'] > 0.2) & (agg['Train'] / agg['Total'] < 0.8)]
    agg[usecol+'Copy'] = agg[usecol]

    train[usecol] = (pd.merge(train[[usecol]], 
                              agg[[usecol, usecol+'Copy']], 
                              on=usecol, how='left')[usecol+'Copy']
                     .replace(np.nan, 0).astype('int').astype('category'))

    test[usecol]  = (pd.merge(test[[usecol]], 
                              agg[[usecol, usecol+'Copy']], 
                              on=usecol, how='left')[usecol+'Copy']
                     .replace(np.nan, 0).astype('int').astype('category'))

    del le, agg_tr, agg_te, agg, usecol
    gc.collect()

In [0]:
(test.isnull().mean()).sort_values(ascending=False)

IsProtected                                          0.003011
Census_SystemVolumeTotalCapacity                     0.000000
Census_IsSecureBootEnabled                           0.000000
Census_ActivationChannel                             0.000000
AppVersion                                           0.000000
Census_InternalBatteryNumberOfCharges                0.000000
Census_ChassisTypeName                               0.000000
Census_ProcessorCoreCount                            0.000000
Census_MDC2FormFactor                                0.000000
Census_PrimaryDiskTypeName                           0.000000
AVProductStatesIdentifier                            0.000000
OrganizationIdentifier                               0.000000
Census_PowerPlatformRoleName                         0.000000
AVProductsInstalled                                  0.000000
Wdft_IsGamer                                         0.000000
SmartScreen                                          0.000000
Census_I

In [0]:
for column in ['IsProtected']:
    train[column].fillna(train[column].mode()[0], inplace=True)

In [0]:
gc.collect()

129

In [0]:
for column in ['IsProtected']:
    test[column].fillna(test[column].mode()[0], inplace=True)

In [0]:
train_ids = train.index
test_ids  = test.index

In [0]:
del train['MachineIdentifier'], test['MachineIdentifier']
gc.collect()

143

In [0]:
ohe = OneHotEncoder(categories='auto', sparse=True, dtype='uint8').fit(train)
gc.collect()

#Transform data using small groups to reduce memory usage
m = 100000
train = vstack([ohe.transform(train[i*m:(i+1)*m]) for i in range(train.shape[0] // m + 1)])
test  = vstack([ohe.transform(test[i*m:(i+1)*m])  for i in range(test.shape[0] // m +  1)])
save_npz('train.npz', train, compressed=True)
save_npz('test.npz',  test,  compressed=True)

In [0]:
del ohe, train, test
gc.collect()

1344

In [0]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, trainy)

lgb_test_result  = np.zeros(test_ids.shape[0])

In [0]:
counter = 0
gc.collect()

0

In [0]:
!pip install lightgbm --install-option=--gpu

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:207: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [0]:
for train_index, test_index in skf.split(train_ids, trainy):
    
    print('Fold {}\n'.format(counter + 1))
    
    train = load_npz('train.npz')
    X_fit = vstack([train[train_index[i*m:(i+1)*m]] for i in range(train_index.shape[0] // m + 1)])
    X_val = vstack([train[test_index[i*m:(i+1)*m]]  for i in range(test_index.shape[0] //  m + 1)])
    X_fit, X_val = csr_matrix(X_fit, dtype='float32'), csr_matrix(X_val, dtype='float32')
    y_fit, y_val = trainy[train_index], trainy[test_index]
    
    del train
    gc.collect()

    lgb_model = lgb.LGBMClassifier(max_depth=-1,
                                   n_estimators=20000,
                                   learning_rate=0.05,
                                   num_leaves=2**12-1,
                                   colsample_bytree=0.28,
                                   objective='binary',
                                   n_jobs=-1)
    lgb_model.fit(X_fit, y_fit, eval_metric='auc', 
                  eval_set=[(X_val, y_val)], 
                  verbose=100, early_stopping_rounds=100)
    del X_fit, X_val, y_fit, y_val, train_index, test_index
    gc.collect()
    
    test = load_npz('test.npz')
    test = csr_matrix(test, dtype='float32')
    lgb_test_result += lgb_model.predict_proba(test)[:,1]
    #xgb_test_result += xgb_model.predict_proba(test)[:,1]
    counter += 1
    
    del test
    gc.collect()

Fold 1

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.608721	valid_0's auc: 0.727328
[200]	valid_0's binary_logloss: 0.602137	valid_0's auc: 0.732488


In [0]:
gc.collect()

In [0]:
submission=pd.read_csv('/content/drive/My Drive/Microsoft Malware/sample_submission.csv')
submission['HasDetections'] = lgb_test_result / counter
submission.to_csv('lgb_submission.csv', index=False)